# Laboratorio de regresión - 4

|                |   |
:----------------|---|
| **Nombre**     | Paola A. Figueroa Álvarez  |
| **Fecha**      |  6/09/2025 |
| **Expediente** |  751310 |

## Modelos penalizados

Hasta ahora la función de costo que usamos para decidir qué tan bueno es nuestro modelo al momento de ajustar es:

$$ \text{RSS} = \sum_{i=1}^n e_i^2 = \sum_{i=1}^n (y_i - \hat{y_i})^2 $$

Dado que los errores obtenidos son una combinación de sesgo y varianza, puede ser que se sesgue un parámetro para minimizar el error. Esto significa que el modelo puede decidir que la salida no sea una combinación de los factores, sino una fuerte predilección sobre uno de los factores solamente. 

E.g. se quiere ajustar un modelo

$$ \hat{z} = \hat{\beta_0} + \hat{\beta_1} x + \hat{\beta_2} y $$

Se ajusta el modelo y se decide que la mejor decisión es $\hat{\beta_1} = 10000$ y $\hat{\beta_2}=50$. Considera limitaciones de problemas reales:
- Quizás los parámetros son ajustes de maquinaria que se deben realizar para conseguir el mejor producto posible, y que $10000$ sea imposible de asignar.
- Quizás los datos actuales están sesgados y sólo hacen parecer que uno de los factores importa más que el otro.

Una de las formas en las que se puede mitigar este problema es penalizando a los parámetros del modelo, cambiando la función de costo:

$$ \text{RSS}_{L2} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p \hat{\beta_j}^2 $$

El *L2* significa que se está agregando una penalización de segundo orden. Lo que hace esta penalización es que los factores ahora sólo tendrán permitido crecer si hay una reducción al menos proporcional en el error (sacrificamos sesgo, pero reducimos la varianza).

Asimismo, existe la penalización *L1*

$$ \text{RSS}_{L1} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p |\hat{\beta_j}| $$

A las penalizaciones *L2* y *L1* se les conoce también como Ridge y Lasso, respectivamente.

Para realizar una regresión con penalización de Ridge o de Lasso usamos el objeto `Ridge(alpha=?)` o `Lasso(alpha=?)` en lugar de `LinearRegression()` de `sklearn`.

Utiliza el dataset de publicidad (Advertising.csv) y realiza 3 regresiones múltiples:

$$ \text{sales} = \beta_0 + \beta_1 (\text{TV}) + \beta_2 (\text{radio}) + \beta_3 (\text{newspaper}) + \epsilon $$

1. Sin penalización
2. Con penalización L2
3. Con penalización L1

Compara los resultados de los parámetros y sus *p-values*, y los $R^2$ resultantes.

### Insertar librerias

In [26]:
!pip install pandas numpy matplotlib scikit-learn


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python3.12 -m pip install --upgrade pip


In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from scipy import stats 

In [2]:
data=pd.read_csv("/Users/paofigueroa/Documents/sem 5/Lab de aprendizaje estadístico/Advertising.csv")
data.head()

,Unnamed: 0,TV,radio,newspaper,sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9


### Regresión sin penalización

In [3]:
x1=data.TV.values.reshape([-1,1])
x2=data.radio.values.reshape([-1,1])
x3=data.newspaper.values.reshape([-1,1])
y=data.sales

In [4]:
X = data[['TV', 'radio', 'newspaper']]
y = data['sales']

lr = LinearRegression()
lr.fit(X, y)

print("Intercept:", lr.intercept_)
print("Coeficientes:", lr.coef_)
r2_total = r2_score(y, lr.predict(X))
p_value = 1 - r2_total
print("R2 total:", r2_total)

Intercept: 2.9388893694594085
Coeficientes: [ 0.04576465  0.18853002 -0.00103749]
R2 total: 0.8972106381789522


In [7]:
y_mean = np.mean(y)
TSS = np.sum((y - y_mean) ** 2)
RSS = (1-r2_total) * TSS
RSS

np.float64(556.8252629021869)

In [8]:
RSE= np.sqrt(RSS/(len(y)-len(lr.coef_)-1))
RSE

np.float64(1.6855103734147436)

In [16]:
n=len(y)
unos=np.ones([n,1])
X=np.hstack([unos, x1,x2,x3])
var_beta1 = np.linalg.inv(X.T @ X) *RSE**2
var_beta1

array([[ 9.72867479e-02, -2.65727337e-04, -1.11548946e-03,
        -5.91021239e-04],
       [-2.65727337e-04,  1.94573710e-06, -4.47039463e-07,
        -3.26595026e-07],
       [-1.11548946e-03, -4.47039463e-07,  7.41533504e-05,
        -1.78006245e-05],
       [-5.91021239e-04, -3.26595026e-07, -1.78006245e-05,
         3.44687543e-05]])

In [20]:
std_beta = np.sqrt(var_beta1.diagonal())
std_beta

array([0.31190824, 0.0013949 , 0.00861123, 0.00587101])

In [ ]:
# Estadísticos T
t_b0 = lr.intercept_/std_beta[0]
t_b1 = lr.coef_[0]/std_beta[1]
t_b2 = lr.coef_[1]/std_beta[2]
t_b3 = lr.coef_[2]/std_beta[3]


In [25]:
#p_values
p_b0= 2*(1-stats.t.cdf(np.abs(t_b0), df=n-2))
p_b1= 2*(1-stats.t.cdf(np.abs(t_b1), df=n-2))
p_b2= 2*(1-stats.t.cdf(np.abs(t_b2), df=n-2))
p_b3= 2*(1-stats.t.cdf(np.abs(t_b3), df=n-2))
p_b0, p_b1, p_b2, p_b3

(np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.8599132081812857))

### Regresión con penalización L2

In [27]:
from sklearn.linear_model import Ridge

In [45]:
X = data[['TV', 'radio', 'newspaper']]
y = data['sales']

ridge = Ridge(alpha=1.0)
ridge.fit(X, y)
r2_ridge= r2_score(y, ridge.predict(X))
print("Ridge Intercept:", ridge.intercept_)
print("Ridge Coeficientes:", ridge.coef_)
print("Ridge R2:", r2_ridge)

Ridge Intercept: 2.9389674583301453
Ridge Coeficientes: [ 0.04576464  0.1885251  -0.00103629]
Ridge R2: 0.8972106380074802


In [53]:
y_mean = np.mean(y)
TSS_r = np.sum((y - y_mean) ** 2)
RSS_r = (1-r2_ridge) * TSS_r
RSS_r

np.float64(556.825263831076)

In [54]:
RSE_r= np.sqrt(RSS_r/(len(y)-len(ridge.coef_)-1))
RSE_r

np.float64(1.6855103748206175)

In [55]:
n=len(y)
unos=np.ones([n,1])
X=np.hstack([unos, x1,x2,x3])
var_beta2 = np.linalg.inv(X.T @ X) *RSE_r**2
var_beta2

array([[ 9.72867480e-02, -2.65727338e-04, -1.11548946e-03,
        -5.91021240e-04],
       [-2.65727338e-04,  1.94573711e-06, -4.47039464e-07,
        -3.26595027e-07],
       [-1.11548946e-03, -4.47039464e-07,  7.41533506e-05,
        -1.78006246e-05],
       [-5.91021240e-04, -3.26595027e-07, -1.78006246e-05,
         3.44687543e-05]])

In [56]:
std_beta2 = np.sqrt(var_beta2.diagonal())
std_beta2

array([0.31190824, 0.0013949 , 0.00861123, 0.00587101])

In [65]:
# Estadísticos T
t_b0_ridge = ridge.intercept_/std_beta2[0]
t_b1_ridge = ridge.coef_[0]/std_beta2[1]
t_b2_ridge = ridge.coef_[1]/std_beta2[2]
t_b3_ridge = ridge.coef_[2]/std_beta2[3]
t_b0_ridge, t_b1_ridge, t_b2_ridge, t_b3_ridge

(np.float64(9.422538790693183),
 np.float64(32.80862311176511),
 np.float64(21.89292468064322),
 np.float64(-0.17651034836178084))

In [58]:
p_b0_ridge= 2*(1-stats.t.cdf(np.abs(t_b0_ridge), df=n-2))
p_b1_ridge= 2*(1-stats.t.cdf(np.abs(t_b1_ridge), df=n-2))
p_b2_ridge= 2*(1-stats.t.cdf(np.abs(t_b2_ridge), df=n-2))
p_b3_ridge= 2*(1-stats.t.cdf(np.abs(t_b3_ridge), df=n-2))
p_b0_ridge, p_b1_ridge, p_b2_ridge, p_b3_ridge

(np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.8599132082969141))

### Regresión con penalización L1

In [59]:
from sklearn.linear_model import Lasso

In [60]:
lasso = Lasso(alpha=1)
lasso.fit(X, y)
r2_lasso= r2_score(y, lasso.predict(X))
print("Lasso Intercept:", lasso.intercept_)
print("Lasso Coeficientes:", lasso.coef_)
print("Lasso R2:", r2_lasso)

Lasso Intercept: 3.0402155834803786
Lasso Coeficientes: [0.         0.04566142 0.1834644  0.        ]
Lasso R2: 0.8970235728389689


In [61]:
y_mean = np.mean(y)
TSS_l = np.sum((y - y_mean) ** 2)
RSS_l = (1-r2_lasso) * TSS_l
RSS_l

np.float64(557.8386236748455)

In [62]:
RSE_l= np.sqrt(RSS_l/(len(y)-len(lasso.coef_)-1))
RSE_l

np.float64(1.6913636186935774)

In [63]:
n=len(y)
unos=np.ones([n,1])
X=np.hstack([unos, x1,x2,x3])
var_beta3 = np.linalg.inv(X.T @ X) *RSE_l**2
var_beta3

array([[ 9.79636134e-02, -2.67576116e-04, -1.12325040e-03,
        -5.95133227e-04],
       [-2.67576116e-04,  1.95927443e-06, -4.50149708e-07,
        -3.28867288e-07],
       [-1.12325040e-03, -4.50149708e-07,  7.46692670e-05,
        -1.79244711e-05],
       [-5.95133227e-04, -3.28867288e-07, -1.79244711e-05,
         3.47085682e-05]])

In [64]:
std_beta3 = np.sqrt(var_beta3.diagonal())
std_beta3

array([0.3129914 , 0.00139974, 0.00864114, 0.0058914 ])

In [67]:
# Estadísticos T
t_b0_lasso = lasso.intercept_/std_beta3[0]
t_b1_lasso = lasso.coef_[0]/std_beta3[1]
t_b2_lasso = lasso.coef_[1]/std_beta3[2]
t_b3_lasso = lasso.coef_[2]/std_beta3[3]
t_b0_lasso, t_b1_lasso, t_b2_lasso, t_b3_lasso

(np.float64(9.713415866248809),
 np.float64(0.0),
 np.float64(5.284190004117579),
 np.float64(31.141064228953823))

In [68]:
#p_values
p_b0_lasso= 2*(1-stats.t.cdf(np.abs(t_b0_lasso), df=n-2))
p_b1_lasso= 2*(1-stats.t.cdf(np.abs(t_b1_lasso), df=n-2))
p_b2_lasso= 2*(1-stats.t.cdf(np.abs(t_b2_lasso), df=n-2))
p_b3_lasso= 2*(1-stats.t.cdf(np.abs(t_b3_lasso), df=n-2))
p_b0_lasso, p_b1_lasso, p_b2_lasso, p_b3_lasso

(np.float64(0.0),
 np.float64(1.0),
 np.float64(3.316329884039959e-07),
 np.float64(0.0))

Observaciones de lo que pasa con alpha. 
Para los nuevos tipos de regresión, Ridge y lasso, si el alpha es más grande, los coeficientes, los hace más pequeños y puede reducir el R². Por el otro lado, si alpha es más pequeño, se penaliza menos, por lo que los coeficientes se parecen más a la regresión sin penalización y R2 aumenta. 

Los tres modelos muestran un desempeño muy similar en términos de R², con valores cercanos a 0.897, lo que indica que explican aproximadamente el 89.7% de la variabilidad de la variable dependiente. 